In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
from typing import Dict, List, Optional
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec14.txt'

In [2]:
def split_val(x):
    amt, name = x.split(' ')
    amt = int(amt)
    return name, amt

def split_formula(x):
    return x.split(' => ')

def split_lhs(x):
    return x.split(', ')

def process_formula(x):
    left, right = split_formula(x)
    output = split_val(right)
    inputs = [split_val(y) for y in split_lhs(left)]
    return inputs, output

In [3]:
create_dict = {}
for formula in INPUT_PATH.read_text().strip().split('\n'):
    inputs, output = process_formula(formula)
    create_dict[output[0]] = (output[1], inputs)

In [4]:
def ore_for_fuel(num_fuel: int) -> int:
    to_make = deque([('FUEL', num_fuel)])
    available = defaultdict(int)
    num_ore_needed = 0

    while to_make:
        output, output_amt = to_make.popleft()

        # Use extra
        foo = min(available[output], output_amt)
        available[output] -= foo
        output_amt -= foo

        # How many times do we need to run the recipe?
        create_amt, inputs = create_dict[output]
        num_recipes = output_amt // create_amt

        # Remainder must still be accounted for
        num_recipes += 1 if output_amt % create_amt != 0 else 0

        # Stack up things we need
        for input_thing, input_amt in inputs:
            input_amt *= num_recipes
            
            # Use extra
            foo = min(available[input_thing], input_amt)
            available[input_thing] -= foo
            input_amt -= foo

            if input_thing == 'ORE':
                # Found the root
                num_ore_needed += input_amt
            else:
                # Keep going
                to_make.append((input_thing, input_amt))

        # After executing this recipe, we have the remainder available
        available[output] += (num_recipes * create_amt) - output_amt
    return num_ore_needed

In [5]:
print(f'The answer to part 1 is {ore_for_fuel(1)}')

The answer to part 1 is 202617


In [6]:
# To determine how much fuel we can make with 1_000_000_000_000 ore,
# we just do baby step / giant step. First get the max

HAVE_ORE = 1_000_000_000_000
cur_amt = 1
while True:
    
    need_ore = ore_for_fuel(cur_amt)
    if need_ore < HAVE_ORE:
        cur_amt *= 2
    else:
        break

In [7]:
# Now binary search
max_val = cur_amt
min_val = cur_amt // 2
cur_amt = (min_val + max_val) // 2
while True:
    need_ore = ore_for_fuel(cur_amt)
    if need_ore <= HAVE_ORE:
        next_amt = (cur_amt + max_val) // 2
        if next_amt == cur_amt:
            break
        min_val = cur_amt
        cur_amt = next_amt
    else:
        next_amt = (cur_amt + min_val) // 2
        max_val = cur_amt
        cur_amt = next_amt

In [8]:
assert ore_for_fuel(cur_amt) < HAVE_ORE and ore_for_fuel(cur_amt + 1) > HAVE_ORE

In [9]:
print(f'The answer to part 2 is {cur_amt}')

The answer to part 2 is 7863863
